### IMPORT LIBRARIES

In [1]:
import os

In [2]:
import numpy as np
import pandas as pd

In [3]:
import seaborn as sns
from matplotlib import pylab as plt

In [4]:
import warnings
warnings.filterwarnings('ignore')
sns.set_palette(['#0892a5','#2e9b9b','#50a290','#6fa985','#8dad7f','#a9b17e','#c4b383','#dbb68f'])

### GETTING DATA

In [5]:
files = [file for file in os.listdir('C:/Users/DMS/Downloads/ecommerce_sales_analysis/dataset')] 

In [6]:
df = pd.DataFrame()

In [7]:
for i in files:
    data = pd.read_csv('dataset/'+i)
    df = pd.concat([df,data],axis=0)

In [8]:
df.shape

(186850, 6)

### DATA CLEANING

In [9]:
# Exclude header for each dataset inside dataframe
df = df[df['Order ID'] != 'Order ID']

In [10]:
df = df.reset_index()

In [11]:
df = df.drop(columns='index')

In [12]:
df.sample(10)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
76620,150016,AA Batteries (4-pack),1,3.84,01/12/19 10:45,"576 Center St, Los Angeles, CA 90001"
19422,237707,Apple Airpods Headphones,1,150,08/29/19 15:04,"617 4th St, San Francisco, CA 94016"
122380,196109,27in 4K Gaming Monitor,1,389.99,05/25/19 18:19,"759 11th St, Dallas, TX 75001"
116739,173183,Wired Headphones,1,11.99,03/20/19 17:53,"712 Main St, Los Angeles, CA 90001"
171840,275940,AA Batteries (4-pack),1,3.84,10/28/19 11:40,"46 Hickory St, Los Angeles, CA 90001"
52753,317121,USB-C Charging Cable,1,11.95,12/12/19 18:26,"531 West St, San Francisco, CA 94016"
129066,202485,AA Batteries (4-pack),1,3.84,05/25/19 18:01,"406 Church St, Los Angeles, CA 90001"
34444,299582,AAA Batteries (4-pack),1,2.99,12/02/19 22:47,"29 Lakeview St, Los Angeles, CA 90001"
9881,186008,Bose SoundSport Headphones,1,99.99,04/17/19 19:36,"432 Park St, New York City, NY 10001"
142180,283901,AAA Batteries (4-pack),3,2.99,11/19/19 20:05,"758 Meadow St, San Francisco, CA 94016"


In [13]:
# Check for data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186495 entries, 0 to 186494
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          185950 non-null  object
 1   Product           185950 non-null  object
 2   Quantity Ordered  185950 non-null  object
 3   Price Each        185950 non-null  object
 4   Order Date        185950 non-null  object
 5   Purchase Address  185950 non-null  object
dtypes: object(6)
memory usage: 8.5+ MB


In [14]:
# Check null values
df.isna().sum()

Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64

In [15]:
# Check null values
df[df.isna().any(axis=1)]

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1,NaN,NaN,NaN,NaN,NaN,NaN
356,NaN,NaN,NaN,NaN,NaN,NaN
734,NaN,NaN,NaN,NaN,NaN,NaN
1430,NaN,NaN,NaN,NaN,NaN,NaN
1550,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
184825,NaN,NaN,NaN,NaN,NaN,NaN
185087,NaN,NaN,NaN,NaN,NaN,NaN
185690,NaN,NaN,NaN,NaN,NaN,NaN
186196,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Drop null vales
df = df.dropna()

In [17]:
# Check null values
df.isna().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
dtype: int64

### DATA PREPARATION

In [20]:
# Correcting data types

df['Quantity Ordered'] = df['Quantity Ordered'].astype('int64')

df['Price Each'] = df['Price Each'].astype('float')

df['Order Date'] = pd.to_datetime(df['Order Date'])

In [21]:
# Adding new feature

def feature_extraction(data):
    
    # funtction to get the city in the data
    def get_city(address):
        return address.split(',')[1]
    
    # funtction to get the state in the data
    def get_state(address):
        return address.split(',')[2].split(' ')[1]

    # let's get the year data in order date column
    data['Year'] = data['Order Date'].dt.year
    
    # let's get the month data in order date column
    data['Month'] = data['Order Date'].dt.month
    
    # let's get the houe data in order date column
    data['Hour'] = data['Order Date'].dt.hour 
    
    # let's get the minute data in order date column
    data['Minute'] = data['Order Date'].dt.minute 
    
    # let's make the sales column by multiplying the quantity ordered colum with price each column
    data['Sales'] = data['Quantity Ordered'] * data['Price Each'] 
    
    # let's get the cities data in order date column
    data['Cities'] = data['Purchase Address'].apply(lambda x: f"{get_city(x)} ({get_state(x)})") 
    
    return data 

In [22]:
df = feature_extraction(df)

In [23]:
df.sample(5)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Year,Month,Hour,Minute,Sales,Cities
87470,232834,Apple Airpods Headphones,1,150.00,2019-07-04 20:08:00,"822 Adams St, Boston, MA 02215",2019,7,20,8,150.00,Boston (MA)
28567,246469,27in 4K Gaming Monitor,1,389.99,2019-08-07 16:50:00,"54 Forest St, Los Angeles, CA 90001",2019,8,16,50,389.99,Los Angeles (CA)
108614,165395,Google Phone,1,600.00,2019-03-26 14:36:00,"419 Pine St, Los Angeles, CA 90001",2019,3,14,36,600.00,Los Angeles (CA)
88096,233433,AA Batteries (4-pack),2,3.84,2019-07-09 08:54:00,"834 Main St, San Francisco, CA 94016",2019,7,8,54,7.68,San Francisco (CA)
150114,291491,Lightning Charging Cable,1,14.95,2019-11-22 16:47:00,"8 Lake St, New York City, NY 10001",2019,11,16,47,14.95,New York City (NY)
